In [1]:
import os

In [2]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [3]:
# from llama_cpp import Llama
from tqdm import tqdm
import numpy as np
import chromadb
import ollama
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering

In [4]:
# import os

import hdbscan
import numpy as np
import pandas as pd
import plotly.express as px
import requests
from dotenv import load_dotenv
from openai import OpenAI
from umap import UMAP



In [5]:
# load_dotenv()

In [6]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["Sri Lanka Constitution-Sinhala_Suff_Rem.txt"]).load_data()

In [7]:
from llama_index.core.node_parser import (
    SentenceSplitter,
    SemanticSplitterNodeParser
)

In [8]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.core import Settings

embed_model = HuggingFaceEmbedding(
    model_name="NLPC-UOM/SinBERT-large",
    max_length=512
)


No sentence-transformers model found with name NLPC-UOM/SinBERT-large. Creating a new one with mean pooling.
Some weights of RobertaModel were not initialized from the model checkpoint at NLPC-UOM/SinBERT-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("NLPC-UOM/SinBERT-large")
# model = AutoModelForMaskedLM.from_pretrained("NLPC-UOM/SinBERT-large")

In [10]:
test = """වෘත්තීය සමිති පිහිටුවීමේ සහ වෘත්තීසමිතිවලට බැඳීමේ නිදහසක් ව්‍යවස්ථාවෙන් ලබා දී ඇත්තේ වරප්‍රසාද ලත් පිරිසකට පමණක් ද?
ශ්‍රී ලංකා ජනරජයේ පරමාධිපත්‍යය ඇත්තේ විධායක ජනාධිපතිවරයාට ද?
ශ්‍රී ලංකා පුරවැසියෙකු අල්ලස් චෝදනාවකට වරදකරු වී පස් වසරක් ගත වී ඇත.  ඔහුට පාර්ලිමේන්තු මැතිවරණය සඳහා ඡන්දය ලබා දිය හැකි ද? එසේ නොවන්නේ නම් ඊට ඇති නීතිමය බාධාව කුමක් ද?
ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩල ප්‍රධානියා වන්නේ අගමැතිවරයා ද?
ශ්‍රේෂ්ඨාධිකරණයේ යම් විනිශ්චයකාරවරයෙකු පත් කිරීමේ යම් දෝෂයක් ඇති විට ශ්‍රේෂ්ඨාධිකරණයේ පවතින නඩු කටයුත්තක් නිර්බල වන්නේ ද?
වරක් තේරී පත් වී සිටින ජනාධිපතිවරයෙකුට නැවත ජනවරමක් ලබා ගැනීම සඳහා තේරී පත් වීමෙන් පසු අවම වශයෙන් වසර කීයක් ඉක්ම විය යුතු ද?
ආණ්ඩුක්‍රම ව්‍යවස්ථා සභාව සාමාජිකයින් කී දෙනෙකුගෙන් සමන්විත විය යුතු ද?
අමාත්‍ය මණ්ඩලයට අයත් උපරිම ඇමතිවරුන් ගණන සහ අමාත්‍ය මණ්ඩල සාමාජිකයින් නොවන ඇමතිවරුන් සහ නියෝජ්‍ය ඇමතිවරුන්ගේ උපරිම ගණන කොපමණද?
පර්ලිමේන්තුව අවම වශයෙන් වසරකට කීවරක් කැඳවිය යුතු ද?
අධිකරණ සේවා කොමිෂන් සභාවෙහි යම් රැස්වීමක ගණපූරණය සඳහා සාමාජිකයින් කී දෙනෙකු සහභාගී විය යුතුද?
පාර්ලිමේන්තුවෙහි ඡන්ද විමසීමක දී මුලසුන දරන මන්ත්‍රීවරයා ඡන්දය දිය යුත්තේ කිනම් අවස්ථාවන්හි දී ද?
කිසියම් ආණ්ඩුකාරවරයෙකු සිය බලය අනිසි ලෙස භාවිත කරන්නේ නම් ඔහු ධූරයෙන් ඉවත් කළ හැක්කේ කෙසේ ද?
ජනාධිපතිවරණයක හෝ ජනමත විචාරණයක වලංගුභාවය සම්බන්ධයෙන් වන නඩු කටයුත්තක තීරණය ලබා දීමට බලය ඇත්තේ කාට ද?
ආණ්ඩුක්‍රම ව්‍යවස්ථා සභාවේ සංයුතිය කුමක් ද?",
ව්‍යවස්ථානුකූලව විගණකාධිපති ධුරය හිස් විය හැකි අවස්ථා කවරේද?"""

In [11]:
# embed_model.get_text_embedding('ශ්‍රී ලංකා ජනරජයේ පරමාධිපත්‍යය ජනතාව කෙරෙහි පිහිටා ඇත්තේ ය.')

In [12]:
base_splitter = SentenceSplitter(chunk_size=512)

In [13]:
# base_splitter(documents)

In [14]:
splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model, 
    # sentence_splitter=base_splitter
)


In [15]:
type(documents[0])

llama_index.core.schema.Document

In [16]:
doc_id_to_document = {doc.id_: doc for doc in documents}
doc_id_to_document

{'dacd091c-4914-4596-a558-4d34049ff044': Document(id_='dacd091c-4914-4596-a558-4d34049ff044', embedding=None, metadata={'file_path': 'Sri Lanka Constitution-Sinhala_Suff_Rem.txt', 'file_name': 'Sri Lanka Constitution-Sinhala_Suff_Rem.txt', 'file_type': 'text/plain', 'file_size': 811078, 'creation_date': '2024-11-01', 'last_modified_date': '2024-11-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='ශ්\u200dර ලං ප්\u200dරජාතාන්ත්\u200dරික සමාජවාද ජනරජ ආණ්ඩුක්\u200dර ව්\u200dයවස්ථාව\r\n(2022 ඔක්තෝබර මස 31 වැ දි දක සංශෝධිතයි)\r\n(විසිඑක සංශෝධන දක සංශෝධ අන්තර්ගත කර ලද)\r\nප්\u200dරතිශෝධිත මුද්\u200dරණ 2023\r\nපාර්ලිමේන්ත මහ ල කාර්ය මගි ප්\u200dරකාශිතයි.\r\n\r\nශ්\u200dර ලං පාර්ලිමේන්ත ව්\u200dයවස්ථාද ස දෙපාර්තමේන්ත ප කෙටුම්ප කාර්යාංශ විසි සංස්කරණ කර  ම නි 

In [17]:
doc_id_to_document.keys()

dict_keys(['dacd091c-4914-4596-a558-4d34049ff044'])

In [18]:
nodes = base_splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
for index, i in enumerate(nodes):
    # if len(i.get_content()) > 2000:
    #     print(len(i.get_content()))
    print("Index : ", index)
    print(f"Chunk Length : {len(i.get_content())}")
    print(i)
    print("*"*100)

Index :  0
Chunk Length : 207
Node ID: f0740958-75ac-4f9d-8acf-2ce41b97b281
Text: ශ්‍ර ලං ප්‍රජාතාන්ත්‍රික සමාජවාද ජනරජ ආණ්ඩුක්‍ර ව්‍යවස්ථාව
(2022 ඔක්තෝබර මස 31 වැ දි දක සංශෝධිතයි)  (විසිඑක සංශෝධන දක සංශෝධ
අන්තර්ගත කර ලද)  ප්‍රතිශෝධිත මුද්‍රණ 2023  පාර්ලිමේන්ත මහ ල කාර්ය මගි
ප්‍රකාශිතයි.
****************************************************************************************************
Index :  1
Chunk Length : 264
Node ID: 26ce8e51-849d-4164-b1ba-8b64a5ee2731
Text: ශ්‍ර ලං පාර්ලිමේන්ත ව්‍යවස්ථාද ස දෙපාර්තමේන්ත ප කෙටුම්ප කාර්යාංශ
විසි සංස්කරණ කර  ම නි න ප්‍රතිශෝධිත මුද්‍රණ මගි ආණ්ඩුක්‍ර ව්‍යවස්ථ
විසිඑක සංශෝධන දක පාර්ලිමේන්ත විසි වරි වර සංශෝධන කර  ශ්‍ර ලං
ප්‍රජාතාන්ත්‍රික සමාජවාද ජනරජ ආණ්ඩුක්‍ර ව්‍යවස්ථ නැවත ප්‍රකාශ ප කර
ලැබේ.
****************************************************************************************************
Index :  2
Chunk Length : 270
Node ID: 76c56a45-0618-4b6d-be85-9dfdb091e72d
Text: අදාළ පිට අග ඇ සටහ මගි ආණ්ඩුක්‍ර ව්‍යවස්ථ සංශෝධන කළ ඒ ඒ සංශෝධ දැ
වේ.    I  පරිච්ඡ

In [20]:
# client = chromadb.Client()
# collection = client.create_collection(name="docs")

# # store each document in a vector embedding database
# for i, d in enumerate(documents):
#   response = ollama.embeddings(model="mxbai-embed-large", prompt=d)
#   embedding = response["embedding"]
#   collection.add(
#     ids=[str(i)],
#     embeddings=[embedding],
#     documents=[d]
#   )

In [21]:
chunks = []

In [22]:
# for node in tqdm(nodes):

#     chunk = {
#         "text" : node.text,
#         "embedding" : embed_model.get_text_embedding(node.text)
#         # llm.create_embedding(node.text)["data"][0]["embedding"]
#         # 
#     }
#     chunks.append([chunk["text"], chunk["embedding"]])

In [23]:
for node in tqdm(nodes[:10]):

    word_embs = embed_model.get_text_embedding_batch(node.text.split(" "))

    # for word in node.text.split(" "):
    #     word_emb = embed_model.get_text_embedding(word)
    #     word_embs.append(word_emb)

    chunk = {
        "text" : node.text,
        "embedding" : word_embs
        # llm.create_embedding(node.text)["data"][0]["embedding"]
        # 
    }
    chunks.append([chunk["text"], chunk["embedding"]])

  0%|          | 0/10 [00:00<?, ?it/s]C:\Users\nawee\AppData\Roaming\Python\Python39\site-packages\transformers\models\roberta\modeling_roberta.py:370: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 10/10 [00:01<00:00,  8.87it/s]


In [24]:
chunks[0][0]

'ශ්\u200dර ලං ප්\u200dරජාතාන්ත්\u200dරික සමාජවාද ජනරජ ආණ්ඩුක්\u200dර ව්\u200dයවස්ථාව\r\n(2022 ඔක්තෝබර මස 31 වැ දි දක සංශෝධිතයි)\r\n(විසිඑක සංශෝධන දක සංශෝධ අන්තර්ගත කර ලද)\r\nප්\u200dරතිශෝධිත මුද්\u200dරණ 2023\r\nපාර්ලිමේන්ත මහ ල කාර්ය මගි ප්\u200dරකාශිතයි.'

In [25]:
data = [chunks[0][1]]

# Convert the list of lists to a NumPy array
array = np.array(data)

# Calculate the mean along the first axis (rows)
mean_array = np.mean(array, axis=0)

print(mean_array.shape)

(27, 768)


In [26]:
# matrix = np.array(chunks[0][1])

# for id1, i in enumerate(matrix):
#     for id2, j in enumerate(matrix):
#         print(round(cosine_similarity_mean(i, j), 3), end=",")
#     print()

In [27]:
hdb = hdbscan.HDBSCAN(min_samples=3, min_cluster_size=3).fit(data[0])

In [44]:
umap = UMAP(n_components=1, random_state=42, n_neighbors=5, min_dist=0.1)

df_umap = (
    pd.DataFrame(umap.fit_transform(np.array(data[0])), columns=['x'])
    .assign(cluster=lambda df: hdb.labels_.astype(str))
    .query('cluster != "-1"')
    .sort_values(by='cluster')
)

fig = px.scatter(df_umap, x='x', color='cluster')
fig.show()

c:\Users\nawee\AppData\Local\anaconda3\envs\SinRag\lib\site-packages\umap\umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [29]:
np.mean([np.mean(data[0][0:10]),np.mean(data[0][10:20]),np.mean(data[0][20:30])])

-0.0001518101363115693

In [30]:
np.mean(data)

-0.0001520967777548921

In [31]:
def cosine_similarity_mean(e1, e2):
#   v1 = chunk
#   v2 = question
  if type(e1) != np.ndarray:
    v1 = np.array(v1)

  if type(e2) != np.ndarray:
    v2 = np.array(e2)


  """
  Calculates the cosine similarity between two vectors.

  Args:
      v1: A numpy array representing the first vector.
      v2: A numpy array representing the second vector.

  Returns:
      A float value between 0 and 1 representing the cosine similarity.
  """
  dot_product = np.dot(e1, e2)
  magnitude1 = np.linalg.norm(e1)
  magnitude2 = np.linalg.norm(e2)
  if magnitude1 == 0 or magnitude2 == 0:
    # Avoid division by zero
    return 0
  return dot_product / (magnitude1 * magnitude2)

In [32]:
def cosine_similarity(chunk, question):
  v1 = chunk
  v2 = question
  if type(v1) != np.ndarray:
    v1 = np.array(v1[1])

  if type(v2) != np.ndarray:
    v2 = np.array(v2)


  """
  Calculates the cosine similarity between two vectors.

  Args:
      v1: A numpy array representing the first vector.
      v2: A numpy array representing the second vector.

  Returns:
      A float value between 0 and 1 representing the cosine similarity.
  """
  dot_product = np.dot(v1, v2)
  magnitude1 = np.linalg.norm(v1)
  magnitude2 = np.linalg.norm(v2)
  if magnitude1 == 0 or magnitude2 == 0:
    # Avoid division by zero
    return 0
  return dot_product / (magnitude1 * magnitude2)

In [33]:
def query_chunks(prompt, embeddings, k=5):
    # embeddings = [i.page_content for i in embeddings]
    chunks = []
    for i in embeddings:
        cs = cosine_similarity(i, prompt)
        chunks.append([i[0], i[1], cs])
    sorted_chunks = sorted(chunks, key=lambda x: x[2], reverse=True)

    return sorted_chunks[:k]

In [34]:
def get_formatted_prompt(question_text):
    k=20
    while True:
        question =  embed_model.get_text_embedding(question_text)
        # llm.create_embedding(question_text)["data"][0]["embedding"] 
        queried = query_chunks(prompt=question, embeddings=chunks, k=k)

        prompt = ""

        for text in queried:
            prompt = prompt + " " + text[0]


        formatted_prompt = f"""
                You are an AI assistant. You can understand English but you can only respond in a language that can be understood by Sri Lankans. Your goal is to answer questions based on the constitution of Sri Lanka based on the following context provided:


                {prompt}


                Answer the following question using only the context given above. Do not include your opinions. Keep the answer clear. Give reasons for yes or no questions. Keep answers short:


                {question_text}


                """
        if len(formatted_prompt) > 5000:
            k = k -1
        else:
            break

        # break
    
    return formatted_prompt

In [35]:
eval = ["වෘත්තීය සමිති පිහිටුවීමේ සහ වෘත්තීසමිතිවලට බැඳීමේ නිදහසක් ව්‍යවස්ථාවෙන් ලබා දී ඇත්තේ වරප්‍රසාද ලත් පිරිසකට පමණක් ද?",
"ශ්‍රී ලංකා ජනරජයේ පරමාධිපත්‍යය ඇත්තේ විධායක ජනාධිපතිවරයාට ද?",
"ශ්‍රී ලංකා පුරවැසියෙකු අල්ලස් චෝදනාවකට වරදකරු වී පස් වසරක් ගත වී ඇත.  ඔහුට පාර්ලිමේන්තු මැතිවරණය සඳහා ඡන්දය ලබා දිය හැකි ද? එසේ නොවන්නේ නම් ඊට ඇති නීතිමය බාධාව කුමක් ද?",
"ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩල ප්‍රධානියා වන්නේ අගමැතිවරයා ද?",
"ශ්‍රේෂ්ඨාධිකරණයේ යම් විනිශ්චයකාරවරයෙකු පත් කිරීමේ යම් දෝෂයක් ඇති විට ශ්‍රේෂ්ඨාධිකරණයේ පවතින නඩු කටයුත්තක් නිර්බල වන්නේ ද?",
"වරක් තේරී පත් වී සිටින ජනාධිපතිවරයෙකුට නැවත ජනවරමක් ලබා ගැනීම සඳහා තේරී පත් වීමෙන් පසු අවම වශයෙන් වසර කීයක් ඉක්ම විය යුතු ද?",
"ආණ්ඩුක්‍රම ව්‍යවස්ථා සභාව සාමාජිකයින් කී දෙනෙකුගෙන් සමන්විත විය යුතු ද?",
"අමාත්‍ය මණ්ඩලයට අයත් උපරිම ඇමතිවරුන් ගණන සහ අමාත්‍ය මණ්ඩල සාමාජිකයින් නොවන ඇමතිවරුන් සහ නියෝජ්‍ය ඇමතිවරුන්ගේ උපරිම ගණන කොපමණද?",
"පර්ලිමේන්තුව අවම වශයෙන් වසරකට කීවරක් කැඳවිය යුතු ද?",
"අධිකරණ සේවා කොමිෂන් සභාවෙහි යම් රැස්වීමක ගණපූරණය සඳහා සාමාජිකයින් කී දෙනෙකු සහභාගී විය යුතුද?",
"පාර්ලිමේන්තුවෙහි ඡන්ද විමසීමක දී මුලසුන දරන මන්ත්‍රීවරයා ඡන්දය දිය යුත්තේ කිනම් අවස්ථාවන්හි දී ද?",
"කිසියම් ආණ්ඩුකාරවරයෙකු සිය බලය අනිසි ලෙස භාවිත කරන්නේ නම් ඔහු ධූරයෙන් ඉවත් කළ හැක්කේ කෙසේ ද?",
"ජනාධිපතිවරණයක හෝ ජනමත විචාරණයක වලංගුභාවය සම්බන්ධයෙන් වන නඩු කටයුත්තක තීරණය ලබා දීමට බලය ඇත්තේ කාට ද?",
"ආණ්ඩුක්‍රම ව්‍යවස්ථා සභාවේ සංයුතිය කුමක් ද?",
"ව්‍යවස්ථානුකූලව විගණකාධිපති ධුරය හිස් විය හැකි අවස්ථා කවරේද?",
"ශ්‍රී ලංකාවෙහි ව්‍යවස්ථාව අනුව අගවිනිසුරුවරයා ලෙස පත් කරනු ලැබූ තැනැත්තාට රාජ්‍යයේ කවරහෝ ඉහළ නිලධාරියෙකු ඉදිරියෙහි දිවුරුම් දීමට ඉඩ දී ඇත්තේ ඇයි? පැහැදිලි කරන්න.",
"ශ්‍රී ලංකාවෙහි ව්‍යවස්ථාව අනුව ජනාධිපතිවරයාට කිසියම් මන්ත්‍රීවරයෙකු අමාත්‍ය මණ්ඩල සාමාජිකයෙක් නොවන අමාත්‍යවරයෙක් ලෙස පත් කිරීමට නොහැකිය. මෙය සත්‍යයක් ද?",
"ශ්‍රී ලංකාවෙහි ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට රජයේ නිලධාරීන් පත් කිරීමට, මාරු කිරීමට හෝ සේවයෙන් පහ කිරීමට විධිවිධාන සැපයීමට බලය නැත. මෙය සාධනීය තත්ත්වයක් ද?",
"පළාත් පාලන ආයතන සහ වෙනත් අධිකාරීන්ට බදු සහ වරිපනම් නියම කිරීමේ බලය ශ්‍රී ලංකා ආණ්ඩුක්‍රම ව්‍යවස්ථාවෙන් සලසා තිබේ. මෙය සාධනීය තත්ත්වයක් ද?",
"ශ්‍රී ලංකා ආණ්ඩුක්‍රම ව්‍යවස්ථාව අනුව රාජ්‍ය සේවා කොමිෂන් සභාවට රාජ්‍ය පරිපාලන ඇමතිවරයා විසින් සාමාජිකයින් පත් කරනු ලබයි. අගමැතිවරයා ඊට විරුද්ධ නම් ඔහු එම සාමාජිකයින් පත් කරන්නේ කෙසේ ද?"]

In [36]:
x = embed_model.get_text_embedding(eval[1])

In [37]:
cosine_similarity(chunks[2], x)

array([0.08980215, 0.08445137, 0.08837033, 0.07721488, 0.08996001,
       0.08708226, 0.10026513, 0.10427136, 0.07924516, 0.08676717,
       0.08812664, 0.08812664, 0.08514698, 0.06938378, 0.08577304,
       0.08199941, 0.09471843, 0.07811154, 0.08234303, 0.11221705,
       0.09047109, 0.08571375, 0.09555172, 0.08551174, 0.09979978,
       0.11273392, 0.07764821, 0.07935318, 0.09047109, 0.09209005,
       0.09047109, 0.07428138, 0.11273392, 0.07764821, 0.0898235 ,
       0.07726255, 0.07930049, 0.08079467, 0.07428138, 0.0898235 ,
       0.08180239, 0.06817786])

In [38]:
for i in query_chunks(embed_model.get_text_embedding(eval[0]), chunks, 10):
    print(i[0])
    print("*"*100)

ValueError: operands could not be broadcast together with shapes (58,) (53,) 

In [ ]:
get_formatted_prompt(eval[0])

ValueError: operands could not be broadcast together with shapes (58,) (53,) 

In [ ]:
answers = {
    
}

In [ ]:
prompt = """14. (1) සෑම පුරවැසියකුට ම
(අ) භාෂණයේ නිදහසට සහ ප්‍රකාශනය ඇතුළු අදහස් පළ කිරීමේ නිදහසට;
(ආ) සාමකාමීව රැස්වීමේ නිදහසට;
(ඇ) සමාගමයේ නිදහසට;
(ඈ) වෘත්තීය සමිති පිහිටුවීමේ සහ වෘත්තීය සමිතිවලට බැඳීමේ නිදහසට;
(ඉ) එකලාව හෝ අන් අය හා සමග, ප්‍රසිද්ධියේ හෝ පෞද්ගලිකව තම ආගම, ලබ්ධිය ඇදහීමෙන්, පිළිපැදීමෙන්, ප්‍රගුණ ඉගැන්වීමෙන් ප්‍රකාශ කිරීමේ නිදහසට; හෝ විශ්වාසය කිරීමෙන් සහ


( ඊ) එකලාව හෝ අන් අය හා සමග හෝ ස්වකීය සංස්කෘතිය භුක්ති විඳීමේ හා වැඩි දියුණු කිරීමේ නිදහසට සහ ස්වකීය භාෂාව භාවිත කිරීමේ නිදහසට;
(උ) එකලාව හෝ අන් අය හා සමඟ යම් නීත්‍යනුකූල රැකියාවක, වෘත්තියක, කර්මාන්තයක, වෙළෙඳ ව්‍යාපාරයක හෝ ව්‍යවසායක නියුක්ත වීමේ නිදහසට;
(ඌ) ශ්‍රී ලංකාව තුළ යාම් ඊම් නිදහසට සහ අභිමත ස්ථානයක වාසය කිරීමේ නිදහසට; සහ
(එ) ශ්‍රී ලංකාවට පෙරළා පැමිණීමේ නිදහසට,
හිමිකම ඇත්තේ ය.
(2) වෙනත් යම් කිසි රටක පුරවැසියකු ව නොසිටිය දී ආණ්ඩුක්‍රම ව්‍යවස්ථාව ක්‍රියාත්මක වීම ආරම්භවන තෙක් ශ්‍රී ලංකාවෙහි ස්ථිරව සහ නීත්‍යනුකූලව පදිංචිව සිටියා වූ සහ එසේ දිගට ම පදිංචිව සිටින්නා වූ තැනැත්තකුටද, ආණ්ඩුක්‍රම ව්‍යවස්ථාව ක්‍රියාත්මක වීම ආරම්භවීමේ සිට අවුරුදු දහයක කාලයක් තුළ මේ ව්‍යවස්ථාවේ (1) වන අනුව්‍යවස්ථාවෙන් ප්‍රකාශ කොට පිළිගනු ලැබ ඇති අයිතිවාසිකම්වලට හිමිකම ඇත්තේ ය.
4"""

In [ ]:
question_text = "වෘත්තීය සමිති පිහිටුවීමේ සහ වෘත්තීසමිතිවලට බැඳීමේ නිදහසක් ව්‍යවස්ථාවෙන් ලබා දී ඇත්තේ වරප්‍රසාද ලත් පිරිසකට පමණක් ද?"

In [ ]:
formatted_prompt = f"""
                You are an AI assistant. You can understand English but you can only respond in a language that can be understood by Sri Lankans. Your goal is to answer questions based on the constitution of Sri Lanka based on the following context provided:'


                {prompt}


                Answer the following question using only the context given above. Do not include your opinions. Keep the answer clear. Give reasons for yes or no questions. Keep answers short:


                {question_text}

                """

In [ ]:
get_formatted_prompt(eval[0]).split(".")

In [ ]:
for i in eval:
  print(i, " : ")
  print(get_formatted_prompt(i))
  response = ollama.chat(model='gemma2:27b-instruct-q4_0',
                        stream=True,
                        messages=[
                                  {
                                    'role': 'user',
                                    'content': get_formatted_prompt(i)
                                  }
                                ]
                        )
  for chunk in response:
    print(chunk['message']['content'], end='', flush=True)


In [53]:
!python query_data.py "ශ්‍රී ලංකා ජනරජයේ පරමාධිපත්‍යය ඇත්තේ විධායක ජනාධිපතිවරයාට ද?"

**************************************************************************************************** [-1.9222092628479004, 0.2527802884578705, 1.323632001876831, -2.0851285457611084, 1.3605414628982544, 1.2102975845336914, -0.6250230669975281, 1.002778172492981, -0.6561558842658997, -0.1969861388206482, -0.34234941005706787, 2.1556105613708496, 2.3355743885040283, 1.3296946287155151, -1.3000023365020752, -0.556963324546814, -0.1569111943244934, 0.8825797438621521, 1.8994733095169067, -1.3919997215270996, -1.0400402545928955, -0.3414720296859741, 0.7206014394760132, -0.4062706232070923, 1.271249532699585, -0.5708876252174377, 0.5913093090057373, -0.7616995573043823, -0.7805213928222656, 0.9480284452438354, -1.6910022497177124, 1.7306241989135742, -1.0044865608215332, -0.32688742876052856, 1.9651833772659302, -2.6666369438171387, 0.4207661747932434, 2.0540931224823, 1.8800729513168335, 1.7526832818984985, -2.7500603199005127, 0.19871346652507782, 1.6487599611282349, -0.8105428218841553, 

c:\Users\nawee\SinhalaTokenizer\app\query_data.py:2: LangChainDeprecationWarning: Importing Chroma from langchain.vectorstores is deprecated. Please replace deprecated imports:

>> from langchain.vectorstores import Chroma

with new imports of:

>> from langchain_community.vectorstores import Chroma
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.vectorstores.chroma import Chroma
c:\Users\nawee\SinhalaTokenizer\app\get_embedding_function.py:12: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(
c:\Users\nawee\SinhalaTokenizer\app


Human: 
                You are an AI assistant. You can understand English but you can only respond in a language that can be understood by Sri Lankans. Your goal is to answer questions based on the constitution of Sri Lanka based on the following context provided:'


                        . 113.         . 113 .

---

1  ,   .

---

(8)                                                                                                                                                                                                   ,                                                       ,                                                               ,                              . 162  
        
                                                            
(9)                                                                                                                                                                                ,                                                      